# The Higgs Boson ML Challenge using MLaaS4HEP



> Install all requirements for MLaaS4HEP and to tackle the challenge



In [ ]:
%matplotlib notebook
%matplotlib inline

In [ ]:
!pip install uproot
!pip install awkward

> Installing ROOT

In [ ]:
!wget https://github.com/palamatt95/HEP-ML/releases/download/ROOT/ROOT.tar.zip
!unzip /content/ROOT.tar.zip
!tar -xf  ROOT.tar
!apt-get install git dpkg-dev cmake g++ gcc binutils libx11-dev libxpm-dev libxft-dev libxext-dev tar gfortran subversion
import sys
sys.path.append("/content/root_build/")
sys.path.append("/content/root_build/bin/")
sys.path.append("/content/root_build/include/")
sys.path.append("/content/root_build/lib/")
import ctypes
ctypes.cdll.LoadLibrary('/content/root_build/lib//libCore.so')
ctypes.cdll.LoadLibrary('/content/root_build/lib//libThread.so')
ctypes.cdll.LoadLibrary('/content/root_build/lib//libTreePlayer.so')

# Before starting, follow the below steps.


## Step 1: Create your Kaggle API Token

- Go to your Kaggle profile and click on Edit Public Profile.
- Scroll the page until API section and click on Create New API Token button.
- A file named kaggle.json will get downloaded

## Step 2: Upload kaggle.json to Google Drive

- Create a folder named "Kaggle" in your Google Drive
- Upload your downloaded kaggle.json file to the created folder

## Now we proceed with the download of the dataset in our Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"

In [ ]:
%cd /content/drive/MyDrive/Kaggle/

In [ ]:
!kaggle competitions download -c higgs-boson

In [ ]:
import zipfile
for file in os.listdir():
    if file.endswith(".zip"):
        with zipfile.ZipFile(file, "r") as zip_file:
            zip_file.extractall()
        os.remove(file)

In [ ]:
for file in os.listdir():
    if file.endswith(".zip"):
        with zipfile.ZipFile(file, "r") as zip_file:
            zip_file.extractall()
        os.remove(file)

**Importing all the required Libraries**

> As you can see, a function "save_fig" has been defined that will allow you to save the images that will be generated

In [ ]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import metrics
from tensorflow import keras
import pickle
import torch

PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "Images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="pdf", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

> We start by opening the file containing the training set and inspecting it

In [ ]:
training = pd.read_csv("/content/drive/MyDrive/Kaggle/training.csv")

In [ ]:
print(training.columns.tolist())
print(len(training.columns.tolist()))
training.shape

**Removing Weights and EventId from the features of the training dataset**

In [ ]:
training.drop(['EventId', 'Weight'], inplace=True, axis=1)
training.shape

**Setting the labels: Signal (s) = 1 and Backgroung (b) = 0**

In [ ]:
training.loc[training["Label"] == "s", "Label"] = 1
training.loc[training["Label"] == "b", "Label"] = 0
training["Label"].value_counts()
training.head()

> And split the Dataset in two part: one containing signal events and the other one containing background

In [ ]:
training_sig = training[training['Label'] == 1]
print(len(training_sig))
training_bkg = training[training['Label'] == 0]
print(len(training_bkg))

In [ ]:
training

Now we will check the shape of each variable in the Training Set

In [ ]:
plt.rcParams["figure.figsize"] = [30,30]
training.hist()
plt.show()
save_fig('feature_hist')

In [ ]:
plt.rcParams["figure.figsize"] = [10,10]
training["Label"].hist()
plt.show()
save_fig('label_hist')

**Look for correlation between variables using the Correlation Matrix**



In [ ]:
corr_matrix = training.corr()

In [ ]:
import seaborn as sn

fig, ax = plt.subplots(figsize=(30,15))
ax = sn.heatmap(
    corr_matrix, 
    vmin=-1, vmax=1, center=0,
    cmap=sn.diverging_palette(20, 220, n=200),
    square=True
)

ax.set_yticklabels(ax.get_yticklabels(which='major'), fontsize=20)
ax.set_xticklabels(
    ax.get_xticklabels(which='major'),
    fontsize = 20,
    rotation=90,
    horizontalalignment='center'
);
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=30)
save_fig('correlation_matrix')

## **Highly Correlated variables should be removed**
> When the correlation is strong, highly correlated variables do not convey extra information. The following list shows these variables, which will later be removed.
- DER_lep_eta_centrality is highly correlated with DER_prodeta_jet_jet
- DER_mass_jet_jet highly correlated with DER_deltaeta_jet_jet
- DER_prodeta_jet_jet is highly correlated with DER_mass_jet_jet
- PRI_jet_all_pt is highly correlated with DER_sum_pt
- PRI_jet_leading_eta is highly correlated with PRI_jet_leading_pt
- PRI_jet_leading_phi is highly correlated with PRI_jet_leading_eta
- PRI_jet_subleading_eta is highly correlated with PRI_jet_subleading_pt
- PRI_jet_subleading_phi is highly correlated with PRI_jet_subleading_eta
- PRI_jet_subleading_pt is highly correlated with DER_lep_eta_centrality
- PRI_met_sumet is highly correlated with DER_sum_pt

##  **Log-transformation makes visualization better when dealing with skewed variables**

> Log-transformation makes our skewed original data more normal and it improves linearity between our dependent and independent variables.

In [ ]:
import seaborn as sn

select=training[["DER_sum_pt","PRI_met_sumet","Label"]]
sn.pairplot(select, hue="Label").fig.suptitle('Without Log Transformation', y=1.05)
plt.show()

x = select["DER_sum_pt"].apply(np.log)
y = select["PRI_met_sumet"].apply(np.log)
z= select["Label"]

d = {'DER_sum_pt': x, 'PRI_met_sumet': y,'Label':z}
new_ = pd.DataFrame(d)
sn.pairplot(new_, hue="Label").fig.suptitle('With Log Transformation', y=1.05)
plt.show()
save_fig('log_visualisation')


> Below there is a list of variables that could be log-transformed.
- DER_sum_pt
- PRI_met_sumet
- PRI_jet_all_pt
- PRI_jet_subleading_pt
- PRI_jet_leading_pt
- PRI_lep_pt
- PRI_tau_pt
- DER_mass_jet_jet

## **Feature Importance using XGBoost**

> Now the study of feature importance will be carried out. To use this technique, it was decided to use an XGBoost classifier

In [ ]:
X, y = training.iloc[:,:-1], training.iloc[:,-1]

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

model = xgb.XGBClassifier()
model.fit(X,y)

In [ ]:
importances = model.feature_importances_
indices = np.argsort(importances)
plt.figure()
plt.rcParams["figure.figsize"] = [20,10]
plt.barh(X.columns[indices], importances[indices], align="center")

plt.yticks(fontsize=18)
plt.ylim([-1, X.shape[1]])
plt.xticks(fontsize=20)
plt.show()
save_fig('feature_importance')


> As shown in the image above, from the feature importance it can be seen that the importance of variables whose name ends with "-phi" is almost zero and therefore they should be removed>

## **Missing Values**

> It can be seen that there are "out of range" values in the dataset that have been set to -999.0. it is necessary, before using MLaaS4HEP, to handle these values.
Our chosen approach is to replace these missing values with the median as shown below.

In [ ]:
training_sig.replace(-999.000, np.nan, inplace=True)
training_bkg.replace(-999.000, np.nan, inplace=True)

In [ ]:
training_sig = training_sig.fillna(training_sig.median())
training_bkg = training_bkg.fillna(training_bkg.median())

## **Conversion to CSV**

> We will now consider two datasets, one referring to "signal" events and one referring to "background" events, from which the "Label" feature will be removed and that will be converted in CSV format.

In [ ]:
training_sig.drop(['Label'], inplace=True, axis=1)
training_bkg.drop(['Label'], inplace=True, axis=1)

In [ ]:
training_sig.to_csv("training_s.csv", index=False)
training_bkg.to_csv("training_b.csv", index=False)

In [ ]:
%ls

## **From CSV to ROOT data format**

> Since MLaaS4HEP requires the input data to be ROOT files, further conversion is required.

In [ ]:
import ROOT

fileName = "training_s.csv";
rdf = ROOT.RDF.MakeCsvDataFrame(fileName);
rdf.Snapshot("myTree", "training_s.root");

fileName = "training_b.csv";
rdf = ROOT.RDF.MakeCsvDataFrame(fileName);
rdf.Snapshot("myTree", "training_b.root");

> With this last step completed, we can move on to use the MLaaS4HEP framework.

# MLaaS4HEP

 First, it is necessary to do a "git clone" to download the code

In [ ]:
!git clone -b Higgs_challenge https://github.com/lgiommi/MLaaS4HEP.git

In [ ]:
%ls

> Place ourselves in the folder containing the file workflow.py

In [ ]:
%cd MLaaS4HEP/src/python/MLaaS4HEP

> And create a new folder containing the ROOT files that will be used for the challenge

In [ ]:
%mkdir challenge_data

In [ ]:
%cd /content/drive/MyDrive/Kaggle

In [ ]:
%mv -t $PWD/MLaaS4HEP/src/python/MLaaS4HEP/challenge_data training_b.root training_s.root

In [ ]:
%cd /content/drive/MyDrive/Kaggle/MLaaS4HEP/src/python/MLaaS4HEP

## MLaaS4HEP Training Workflow
The MLaaS4HEP Training Workflow is performed by running the workflow.py python
script which takes several argument as input.



- files.txt: stores paths and names of the input ROOT files.

In [ ]:
%cat files.txt

- labels.txt contains the labels of the respective ROOT files, it is used for classification
problems.

In [ ]:
%cat labels.txt

- a python file that contains the definition of the ML model chosen by user

> To address this challenge, we choose four different models with which we will run MLaaS4HEP. The following is the definition of the first model, in which a gradient boosting classifier was defined.

In [ ]:
%cat gradient_boosting.py

- params.json stores parameters on which MLaaS4HEP relies, such as chunk size,
batch size, number of epochs and so on.

> In our case, two different types of params.json are defined depending on the model used.

> In addition, it can be seen that this file contains the names of the variables that we decided to remove from the dataset as suggested by the preliminary analysis and this can be done by entering the name of the variables as the value of the "exclude_branches" key

In [ ]:
%cat params_DT.json

- preproc.json contains the preprocessing operations that the user wants to perform.

> When we talk about preprocessing operations, we refer to operations
that allow the users to manipulate data, i.e.:
- new branches definition,
- application of cuts on branches, both new and existing ones,
- removal of branches that may not be useful for model training.

> In this case, we use preproc.json to apply the logarithmic transformation on the features of the dataset as suggested in the previous analysis.

In [ ]:
%cat preproc.json

## Now we are ready to execute MLaaS4HEP

In [ ]:
!python workflow.py --files=files.txt --labels=labels.txt --model=gradient_boosting.py --params=params_DT.json --preproc=preproc.json --fout=challenge_data/GBModel.pkl

> Definition of the XGBoost Classifier

In [ ]:
%cat XGBoost.py

In [ ]:
!python workflow.py --files=files.txt --labels=labels.txt --model=XGBoost.py --params=params_DT.json --preproc=preproc.json --fout=challenge_data/XGBModel.json

> Definition of a Sequential Neural Network using Keras

In [ ]:
%cat sequential_NN.py

> params_NN.json: this file contains different information than the previous one, such as chunk size and number of epochs

In [ ]:
%cat params_NN.json

In [ ]:
!python workflow.py --files=files.txt --labels=labels.txt --model=sequential_NN.py --params=params_NN.json --preproc=preproc.json --fout=challenge_data/KerasModel.h5

> Definition of a Sequential Neural Network using PyTorch

In [ ]:
%cat clf_torch.py

In [ ]:
!python workflow.py --files=files.txt --labels=labels.txt --model=clf_torch.py --params=params_NN.json --preproc=preproc.json --fout=challenge_data/torch_model.pth

In [ ]:
%ls

# Preparing the Test set for predictions

> Now we open the file containing the test set

In [ ]:
%cd /content/drive/MyDrive/Kaggle

In [ ]:
test_subset = pd.read_csv("/content/drive/MyDrive/Kaggle/test.csv")

In [ ]:
test_subset.info()

> Replace missing values using the median (like in the previous case)

In [ ]:
test_subset.replace(-999.000, np.nan, inplace=True)

In [ ]:
test = test_subset.fillna(test_subset.median())

In [ ]:
test.head()

> Set the variable "EventId" as the index of the Dataframe

In [ ]:
ids = test['EventId']
test.set_index(['EventId'],inplace = True)

> Apply the log-transformation just as was done within MLaaS4HEP using the preproc.json file

In [ ]:
logcolumns = ["DER_sum_pt","PRI_jet_leading_pt","PRI_lep_pt","PRI_tau_pt"]
test[logcolumns]

In [ ]:
test.loc[:, logcolumns] = np.log(test[logcolumns])
test[logcolumns]

In [ ]:
test

> Again, it is necessary to select the features to be excluded

In [ ]:
x_test = test.drop(["PRI_tau_phi",
        "PRI_lep_phi", "PRI_met_phi", "PRI_jet_leading_phi",
        "PRI_jet_subleading_phi", "DER_lep_eta_centrality",
        "DER_mass_jet_jet", "DER_prodeta_jet_jet", "PRI_jet_all_pt",
        "PRI_jet_leading_eta", "PRI_jet_subleading_eta",
        "PRI_jet_subleading_pt", "PRI_met_sumet"], axis=1)

> And we conclude by putting the features in the same order as they were processed by MLaaS4HEP, which arranges them in alphabetical order. In case new variables are defined, as in our case, they will be placed after existing features

In [ ]:
cols = ['DER_deltaeta_jet_jet','DER_deltar_tau_lep','DER_mass_MMC','DER_mass_transverse_met_lep','DER_mass_vis',
       'DER_met_phi_centrality','DER_pt_h','DER_pt_ratio_lep_tau','DER_pt_tot','PRI_jet_num', 'PRI_lep_eta', 
        'PRI_met', 'PRI_tau_eta','DER_sum_pt','PRI_jet_leading_pt','PRI_lep_pt','PRI_tau_pt']


len(cols)

In [ ]:
x_test = x_test[cols]

In [ ]:
x_test

# Creating the submission file

> As a first step, it is necessary to use the models trained by MLaaS4HEP to make inference on the new data.

> The pickle library allows to save and load trained Scikit Learn model

In [ ]:
def load_code(mfile, fname):
    """
    Load function from given python module (file)
    """
    mname = mfile.split('.py')[0].replace('/', '.')
    try:
        mod = __import__(mname, fromlist=['model'])
        func = getattr(mod, fname)
        #print("load {} {} {}".format(mfile, func, func.__doc__))
        return func
    except ImportError:
        traceback.print_exc()
        msg = "Please provide file name with 'def %s' implementation" % fname
        msg += "\nThe file should be available in PYTHONPATH"
        print(msg)
        raise

In [ ]:
%cd /content/drive/MyDrive/Kaggle/MLaaS4HEP/src/python/MLaaS4HEP/

In [ ]:
PTModel = load_code('clf_torch.py', 'model')

In [ ]:
idim = np.shape(x_test)[-1]
PTmodel = PTModel(idim)

In [ ]:
%cd challenge_data/

In [ ]:
PTmodel.load_state_dict(torch.load("torch_model.pth"))
PTmodel.eval()

In [ ]:
KModel = keras.models.load_model("KerasModel.h5")
XGBModel = xgb.XGBClassifier()
XGBModel.load_model('XGBModel.json')
GBModel = pickle.load(open('GBModel.pkl', 'rb'))

In [ ]:
PTpred = PTmodel(x_test.values)
PTpred = PTpred.detach().numpy()

In [ ]:
Kpred = KModel.predict(x_test.values)
Xpred = XGBModel.predict_proba(x_test.values)[:,1]
Gpred = GBModel.predict_proba(x_test.values)[:,1]

> Now let's proceed with creating the CSV to make the submission to Kaggle, but first we need to define the threshold.

> If we consider, for example, a Neural Network, the activation function connected to the output layer will map any variable in a range of values between 0 and 1. So, for binary classification problems, it is necessary to set a threshold which divides the values into two categories: values below the threshold are assigned to the 0 category otherwise to the 1 category.
This threshold is usually set to 0.5 by default, but it may not always be the best solution.

> At the end of the execution of MLaaS4HEP, for each run at the end of the output you can see a "Best Threshold=...": such value corresponds to the best threshold according to the predictions made on the training set. There is a threshold present for each model and it is necessary to enter the values below.
The ones you see written here are the ones obtained from a previous launch.

> Note that K stands for Keras and refers to the neural network, X refers to the XGBoost classifier and G refers to the gradient boosting classifier

In [ ]:
PTthresh = 0.384685

In [ ]:
Kthresh = 0.224096
Xthresh = 0.478793
Gthresh = 0.394601

> Here we are building the submission file for each model

In [ ]:
Xp = np.empty(len(x_test.values), dtype=object)
Xp[Xpred > Xthresh] = 's'
Xp[Xpred <= Xthresh] = 'b'
Xr = np.argsort(Xpred) + 1
XGBsub = pd.DataFrame({"EventId": ids, "RankOrder": Xr, "Class": Xp})
XGBsub

In [ ]:
Gp = np.empty(len(x_test.values), dtype=object)
Gp[Gpred > Gthresh] = 's'
Gp[Gpred <= Gthresh] = 'b'
Gr = np.argsort(Gpred) + 1
GBsub = pd.DataFrame({"EventId": ids, "RankOrder": Gr, "Class": Gp})
GBsub

In [ ]:
Kp = np.empty(len(x_test.values), dtype=object)
Kpred = Kpred.flatten()
Kp[Kpred > Kthresh] = 's'
Kp[Kpred <= Kthresh] = 'b'
Kr = np.argsort(Kpred) + 1
Ksub = pd.DataFrame({"EventId": ids, "RankOrder": Kr, "Class": Kp})
Ksub

In [ ]:
PTp = np.empty(len(x_test.values), dtype=object)
PTpred = PTpred.flatten()
PTp[PTpred > PTthresh] = 's'
PTp[PTpred <= PTthresh] = 'b'
PTr = np.argsort(PTpred) + 1
PTsub = pd.DataFrame({"EventId": ids, "RankOrder": PTr, "Class": PTp})
PTsub

In [ ]:
Ksub.to_csv('submission_NN.csv', index=False)
GBsub.to_csv('submission_GBC.csv', index=False)
XGBsub.to_csv('submission_XGB.csv', index=False)

In [ ]:
PTsub.to_csv('submission_PT.csv', index=False)

> And finally we proceed to upload each submission file to the Kaggle site.

In [ ]:
!kaggle competitions submit higgs-boson -f submission_NN.csv -m "Submission with a NN written in Keras"
!kaggle competitions submit higgs-boson -f submission_GBC.csv -m "Submission with a Gradient Boosting Classifier"
!kaggle competitions submit higgs-boson -f submission_XGB.csv -m "Submission with a XGBoost Classifier"

In [ ]:
!kaggle competitions submit higgs-boson -f submission_PT.csv -m "Submission with a PyTorch Classifier"

Now you can go and check at https://www.kaggle.com/competitions/higgs-boson/leaderboard) what was the score obtained by the models.